Location bug still exists...try to filter out <2 words ones?

-*- coding: utf-8 -*-

"""

Created on Thu Aug 23, 2020

@author: Rohitashwa

@Credits: loren ( https://github.com/jjeffus/gofundme)

"""

In [ ]:
# !pip install selenium
# !pip install bs4
# !pip install requests
# !pip install pytest-shutil

In [2]:
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
from time import sleep
import requests
import re
import shutil # For image download

In [18]:
mydf = pd.read_csv('data/GFM_url_7k.csv', sep = '\t')
headers = ["Url", "Category","Position", "Title", "Location", "Organiser","Amount_Raised", "Goal", "Duration", "Text", "Image"]
mydf = mydf.reindex(columns = headers)
print(mydf.shape)
mydf.head()

(6922, 11)


,Url,Category,Position,Title,Location,Organiser,Amount_Raised,Goal,Duration,Text,Image
0,https://www.gofundme.com/f/face-and-acne-treat...,Medical,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,https://www.gofundme.com/f/donation-for-uterus...,Medical,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,https://www.gofundme.com/f/money-to-buy-hearin...,Medical,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,https://www.gofundme.com/f/ht4vqf-eye-surgery,Medical,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,https://www.gofundme.com/f/tqtuw-self-employment,Medical,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
mydf.iloc[0]

Url              https://www.gofundme.com/f/painless-alternativ...
Category                                                  Business
Position                                                       NaN
Title                                                          NaN
Location                                                       NaN
Organiser                                                      NaN
Amount_Raised                                                  NaN
Goal                                                           NaN
Duration                                                       NaN
Text                                                           NaN
Image                                                          NaN
Name: 0, dtype: object

# Scrape Individual URLs

In [19]:
listInvalidCampaign = []
lstScrapingError = []

def scrape_url(row_index, tid = "mainThread"):
    print("Scraping url %s" %(row_index+1))
    
    single_row = mydf.iloc[row_index]
    url = single_row["Url"]
    category = single_row["Category"]
    position = single_row["Position"]

    page = requests.get(url)
    soup = BeautifulSoup(page.text, 'lxml')
    
    temp_df = pd.DataFrame(columns = headers)
    
#     invalidCampaign = soup.find('div', attrs={'class': 'm-campaign-sidebar-notification m-well m-well--sand'})
    
#     if(invalidCampaign is not None):
#         text_container =  soup.find('meta', attrs={'name': 'description'})
#         print(text_container['content'])
#         listInvalidCampaign.append(row_index)
#         return "Invalid Campaign"
    
    try:
        # Description
        text_container =  soup.find('meta', attrs={'name': 'description'})
        try:
            all_text = text_container['content']
        except:
            all_text = np.nan

        if(all_text is np.nan )or(all_text == "We're sorry, but that campaign cannot be found. Please check the link URL and try again."):
            print(tid+' ('+str(row_index)+') -> '+all_text)
            listInvalidCampaign.append(row_index)
            temp_df = pd.DataFrame(columns = headers)
            return(temp_df)

        # Title
        try:
            title = soup.find("title").text
        except:
            title = np.nan

        # Organiser & Location
        try:
            t = soup.find_all("div",{"class":"p-campaign-members"})[0]
            organiser = t.find_all("div",{"class": "m-person-info-name"})[0].text.replace('\xa0','')
            location = ""
            if(len(t.find_all("div",{"class": "text-small"})) > 3):
                location = t.find_all("div",{"class": "text-small"})[2].text.replace('\xa0','')
            else:
                location = t.find_all("div",{"class": "text-small"})[1].text.replace('\xa0','')
        except:
            organiser = np.nan
            location = np.nan

        # Amount Raised and Goal
        try:
            t = soup.find("div",{"class":"p-campaign-sidebar"})
            amount = t.find("div",{"class": "o-campaign-sidebar-progress-meter m-progress-meter"})
            amountRaised = amount.text.split()[0]
            goal = amount.text.split()[-2]
        except:
            amountRaised = np.nan
            goal = np.nan

        # All Text
        soup.find('div', {'class': "p-campaign-description"})
        text = soup.find('div', {'class': "o-campaign-story mt-three-halves"}).text.replace('\xa0','').replace('\n','')
        text

        # Duration of Fundraising
        try:
            t = soup.find("div",{"class": "p-campaign-description"})
            startDate = ' '.join(t.find("li",{"class": "m-meta-list-item"}).text.split()[1::])
            startDate = (pd.to_datetime(startDate)).date()
            duration = (pd.datetime.now().date() - startDate).days
        except:
            duration = np.nan

        #Image
        try:
            img_container =  soup.find('meta', attrs={'property': 'og:image'})
            image_url = img_container['content']

            img_filename = "data/images/"+image_url.split("/")[-1]

            # Open the url image, set stream to True, this will return the stream content.
            r = requests.get(image_url, stream = True)

            # Check if the image was retrieved successfully
            if r.status_code == 200:
                # Set decode_content value to True, otherwise the downloaded image file's size will be zero.
                r.raw.decode_content = True

                # Open a local file with wb ( write binary ) permission.
                with open(img_filename,'wb') as f:
                    shutil.copyfileobj(r.raw, f)
        except:
            img_filename = np.nan

    #     # FB likes and Shares    
    #     try:
    #         FB_shares_container = soup.find_all("strong", {"class":"js-share-count-text"})
    #         FB_shares = FB_shares_container[0].text.splitlines()
    #         FB_shares = FB_shares[1].replace(" ", "").replace("\xa0", "")
    #     except:
    #         FB_shares = np.nan

    #     try:
    #         hearts_container = soup.find_all("div", {"class":"campaign-sp campaign-sp--heart fave-num"})
    #         hearts = hearts_container[0].text
    #     except:
    #         hearts = np.nan

        temp_row = np.array([[url, category, position, title, location, organiser, amountRaised, goal, duration, text, img_filename]])
        temp_df = pd.DataFrame(temp_row, columns = headers)

        return(temp_df)
    
    except:
        lstScrapingError.append(row_index)
        return tid+' ('+str(row_index)+') -> '+"Scraping Error"

## Multithreaded Version

In [20]:
def scrape_all_urlsV2(params):
    start, end , tid = params
    print("Started Scraping for Thread: "+str(tid))
    
    finaldf = pd.DataFrame(columns = headers)
    for i in range(start, end):
        try:
            sleep(6)
            t = scrape_url(i, tid)
            if(type(t) != str):
                finaldf = pd.concat([finaldf,t], ignore_index = True)
            else:
                print(t)
        except Exception as e:
            print("Thread ID "+str(tid),getattr(e, 'message', repr(e)))
    return finaldf
    
#     for i in range(start, end):
#         try:
#             t = scrape_url(i)
#             temp_df = pd.concat([temp_df,t], ignore_index = True)
#         except:
#             try:
#                 sleep(5)
#                 t = scrape_url(i)
#                 temp_df = pd.concat([temp_df,t], ignore_index = True)
#             except:
#                 print("Thread ID "+str(tid)+": Cant Scrape Row: "+str(i))

In [21]:
#Distribute regions for the multiple threads
def BatchGenerator(start_index, end_index, batch_size):
    tid = 0
    for step in range(start_index,end_index,batch_size):
        tid = tid+1
        end = (step+batch_size) if ((step+batch_size) < end_index) else (end_index)
        yield (step, end, 'Thread_'+str(tid))

In [26]:
for i in BatchGenerator(0, mydf.shape[0], 400):
# for i in BatchGenerator(0, 50, 10):
    print(i)

(0, 400, 'Thread_1')
(400, 800, 'Thread_2')
(800, 1200, 'Thread_3')
(1200, 1600, 'Thread_4')
(1600, 2000, 'Thread_5')
(2000, 2400, 'Thread_6')
(2400, 2800, 'Thread_7')
(2800, 3200, 'Thread_8')
(3200, 3600, 'Thread_9')
(3600, 4000, 'Thread_10')
(4000, 4400, 'Thread_11')
(4400, 4800, 'Thread_12')
(4800, 5200, 'Thread_13')
(5200, 5600, 'Thread_14')
(5600, 6000, 'Thread_15')
(6000, 6400, 'Thread_16')
(6400, 6800, 'Thread_17')
(6800, 6922, 'Thread_18')


In [27]:
# Online Mode
_ = pd.DataFrame(columns = headers)
_.to_csv("data/GFM_data.csv",sep = '\t')


finaldf = pd.DataFrame(columns=headers)

# Multi-Threaded
import time
import concurrent.futures

start_record = 0
end_record = mydf.shape[0]
batch_size = 1000

t0= time.time()

with concurrent.futures.ThreadPoolExecutor(max_workers=20) as executor:
    futures = [executor.submit(scrape_all_urlsV2, param) for param in BatchGenerator(start_record, end_record, batch_size)]
    return_value = [f.result() for f in futures]

    rowCount = 1
    for k in return_value:
        finaldf = pd.concat([finaldf,k], ignore_index = True)
        rowCount = rowCount + batch_size

t1= time.time() -t0

Started Scraping for Thread: Thread_1Started Scraping for Thread: Thread_2

Started Scraping for Thread: Thread_3
Started Scraping for Thread: Thread_4
Started Scraping for Thread: Thread_5
Started Scraping for Thread: Thread_6
Started Scraping for Thread: Thread_7
Scraping url 1001
Scraping url 1
Scraping url 2001
Scraping url 3001
Scraping url 4001
Scraping url 5001
Scraping url 6001
Thread_1 (0) -> We're sorry, but that campaign cannot be found. Please check the link URL and try again.


<ipython-input-19-8885488b2d17>:78: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  duration = (pd.datetime.now().date() - startDate).days


Scraping url 2
Scraping url 5002
Scraping url 1002
Thread_1 (1) -> We're sorry, but that campaign cannot be found. Please check the link URL and try again.
Scraping url 3002
Scraping url 2002
Scraping url 6002
Scraping url 4002
Scraping url 3
Scraping url 5003
Scraping url 4003Scraping url 3003Scraping url 2003Scraping url 6003

Scraping url 1003


Thread_2 (1002) -> We're sorry, but that campaign cannot be found. Please check the link URL and try again.
Scraping url 4
Thread_1 (3) -> We're sorry, but that campaign cannot be found. Please check the link URL and try again.
Scraping url 5004
Scraping url 6004
Scraping url 1004
Scraping url 4004
Scraping url 2004
Scraping url 3004
Scraping url 5
Thread_1 (4) -> We're sorry, but that campaign cannot be found. Please check the link URL and try again.
Scraping url 5005
Scraping url 1005
Scraping url 6005
Scraping url 2005
Scraping url 4005
Scraping url 3005
Scraping url 6
Scraping url 5006
Scraping url 1006
Scraping url 6006
Scraping url 200

Scraping url 59
Scraping url 1058
Scraping url 5059
Scraping url 6059
Scraping url 2059
Scraping url 3059
Scraping url 4059
Scraping url 1059
Scraping url 60
Scraping url 5060
Scraping url 6060
Scraping url 2060
Scraping url 3060
Scraping url 1060
Scraping url 4060
Scraping url 61
Scraping url 5061
Scraping url 6061
Scraping url 2061
Scraping url 4061
Scraping url 1061
Scraping url 3061
Scraping url 62
Scraping url 5062
Scraping url 6062
Scraping url 2062
Scraping url 4062
Scraping url 1062
Scraping url 3062
Scraping url 5063
Scraping url 63
Scraping url 6063
Scraping url 2063
Scraping url 4063
Scraping url 1063
Scraping url 3063
Scraping url 5064
Scraping url 64
Scraping url 6064
Scraping url 2064
Scraping url 4064
Scraping url 1064
Scraping url 3064
Scraping url 5065
Scraping url 65
Scraping url 6065
Scraping url 2065
Scraping url 1065
Scraping url 4065
Scraping url 3065
Scraping url 5066
Scraping url 6066
Scraping url 66
Scraping url 2066
Scraping url 4066
Scraping url 1066
Scraping

Scraping url 3122
Scraping url 1122
Scraping url 2123
Scraping url 6123
Scraping url 5124
Scraping url 4123
Scraping url 123
Scraping url 3123
Scraping url 1123
Scraping url 2124
Scraping url 6124
Scraping url 4124
Scraping url 5125
Scraping url 124
Scraping url 3124
Scraping url 1124
Scraping url 2125
Scraping url 6125
Scraping url 4125
Scraping url 5126
Scraping url 125
Scraping url 3125
Scraping url 1125
Thread_1 (124) -> Scraping Error
Scraping url 2126
Scraping url 6126
Scraping url 4126
Scraping url 5127
Scraping url 3126
Scraping url 1126
Scraping url 126
Scraping url 2127
Scraping url 6127
Scraping url 4127
Scraping url 5128
Scraping url 3127
Scraping url 1127
Scraping url 127
Scraping url 2128
Scraping url 6128
Scraping url 4128
Scraping url 5129
Scraping url 3128
Scraping url 1128
Scraping url 128
Scraping url 2129
Scraping url 6129
Scraping url 5130
Scraping url 4129
Scraping url 3129
Scraping url 1129
Scraping url 129
Scraping url 2130
Scraping url 6130
Scraping url 5131
Sc

...

Scraping url 3694
Scraping url 2698
Scraping url 5695
Scraping url 694
Scraping url 1696
Scraping url 6695
Scraping url 3695
Scraping url 2699
Scraping url 4697
Scraping url 5696
Scraping url 695
Scraping url 1697
Scraping url 6696
Scraping url 3696
Scraping url 2700
Scraping url 4698
Scraping url 5697
Scraping url 696
Scraping url 1698
Scraping url 6697
Scraping url 3697
Scraping url 2701
Scraping url 5698
Scraping url 4699
Scraping url 697
Scraping url 1699
Scraping url 6698
Scraping url 3698
Scraping url 2702
Scraping url 5699
Scraping url 4700
Scraping url 698
Scraping url 1700
Scraping url 6699
Scraping url 3699
Scraping url 2703
Scraping url 5700
Scraping url 4701
Scraping url 1701
Scraping url 699
Scraping url 6700
Scraping url 2704
Scraping url 3700
Scraping url 5701
Scraping url 4702
Scraping url 1702
Scraping url 700
Scraping url 6701
Scraping url 3701
Scraping url 2705
Scraping url 5702
Scraping url 4703
Scraping url 1703
Scraping url 701
Scraping url 6702
Scraping url 2706


Scraping url 3759
Scraping url 6758
Scraping url 4761
Scraping url 758
Scraping url 1761
Scraping url 5760
Scraping url 2764
Scraping url 3760
Scraping url 6759
Scraping url 759
Scraping url 4762
Scraping url 1762
Scraping url 5761
Scraping url 2765
Scraping url 3761
Scraping url 760
Scraping url 6760
Scraping url 4763
Scraping url 1763
Scraping url 5762
Scraping url 2766
Scraping url 3762
Thread_3 (2765) -> Scraping Error
Scraping url 761
Scraping url 4764
Scraping url 6761
Scraping url 1764
Scraping url 5763
Scraping url 2767
Scraping url 3763
Scraping url 762
Scraping url 6762
Scraping url 4765
Scraping url 1765
Scraping url 2768
Scraping url 5764
Scraping url 3764
Scraping url 763
Scraping url 6763
Scraping url 4766
Scraping url 1766
Scraping url 2769
Scraping url 5765
Scraping url 3765
Scraping url 764
Scraping url 6764
Scraping url 4767
Scraping url 1767
Scraping url 5766
Scraping url 2770
Scraping url 3766
Scraping url 765
Scraping url 6765
Scraping url 4768
Scraping url 1768
Sc

Scraping url 822
Scraping url 2828
Scraping url 3823
Scraping url 6822
Thread_7 (6821) -> Scraping Error
Scraping url 4825
Scraping url 1825
Scraping url 5824
Scraping url 823
Scraping url 2829
Scraping url 3824
Scraping url 6823
Scraping url 4826
Scraping url 1826
Scraping url 5825
Scraping url 824
Scraping url 2830
Scraping url 6824
Scraping url 3825
Scraping url 4827
Scraping url 1827
Scraping url 5826
Scraping url 2831
Scraping url 825
Scraping url 3826
Scraping url 6825
Scraping url 4828
Scraping url 1828
Scraping url 5827
Scraping url 2832
Scraping url 826
Scraping url 3827
Scraping url 6826
Scraping url 1829
Scraping url 4829
Scraping url 5828
Scraping url 2833
Scraping url 827
Scraping url 3828
Scraping url 6827
Scraping url 1830
Scraping url 4830
Scraping url 5829
Scraping url 2834
Scraping url 828
Scraping url 3829
Scraping url 6828
Scraping url 1831
Scraping url 5830
Scraping url 4831
Scraping url 2835
Scraping url 829
Scraping url 3830
Scraping url 6829
Scraping url 1832
Sc

Scraping url 5887
Scraping url 3886
Scraping url 2892
Thread_3 (2891) -> Scraping Error
Scraping url 1888
Scraping url 886
Scraping url 4888
Scraping url 6886
Scraping url 5888
Scraping url 3887
Scraping url 2893
Scraping url 1889
Scraping url 887
Scraping url 4889
Scraping url 6887
Scraping url 5889
Scraping url 3888
Scraping url 2894
Scraping url 1890
Scraping url 888
Scraping url 4890
Scraping url 6888
Scraping url 3889
Scraping url 5890
Scraping url 2895
Scraping url 1891
Scraping url 889
Scraping url 4891
Scraping url 6889
Scraping url 3890
Scraping url 5891
Scraping url 2896
Thread_3 (2895) -> Scraping Error
Scraping url 1892
Scraping url 890
Scraping url 4892
Scraping url 6890
Scraping url 3891
Scraping url 5892
Scraping url 2897
Scraping url 1893
Scraping url 891
Scraping url 4893
Scraping url 6891
Scraping url 3892
Scraping url 5893
Scraping url 2898
Scraping url 1894
Scraping url 892
Scraping url 4894
Scraping url 6892
Scraping url 3893
Scraping url 5894
Scraping url 2899
Scr

Scraping url 941
Scraping url 3943
Scraping url 4943
Scraping url 5943
Scraping url 2948
Scraping url 1944
Scraping url 942
Scraping url 3944
Scraping url 4944
Scraping url 1945
Scraping url 5944
Scraping url 2949
Scraping url 943
Scraping url 3945
Scraping url 4945
Scraping url 1946
Scraping url 5945
Scraping url 2950
Scraping url 944
Scraping url 3946
Scraping url 4946
Scraping url 1947
Scraping url 5946
Scraping url 2951
Scraping url 945
Scraping url 3947
Scraping url 4947
Scraping url 1948
Scraping url 5947
Scraping url 2952
Scraping url 946
Scraping url 3948
Scraping url 4948
Scraping url 1949
Scraping url 5948
Scraping url 2953
Scraping url 947
Scraping url 4949
Scraping url 3949
Scraping url 1950
Scraping url 5949
Scraping url 2954
Scraping url 948
Scraping url 3950
Scraping url 4950
Scraping url 1951
Scraping url 2955
Scraping url 949
Scraping url 3951
Scraping url 4951
Scraping url 1952
Scraping url 5950
Scraping url 2956
Scraping url 950
Scraping url 3952
Scraping url 4952
Sc

In [35]:
finaldf.to_csv('data/GFM_data.csv',sep = '\t', index = False)
finaldf

,Url,Category,Position,Title,Location,Organiser,Amount_Raised,Goal,Duration,Text,Image
0,https://www.gofundme.com/f/money-to-buy-hearin...,Medical,0,Fundraiser by Chandan Shivu Kumar : Money to b...,nan,nan,$250,$250,236,"Hello there, I'm a random guy from India who's...",data/images/44793298_1579175168658123_r.jpeg
1,https://www.gofundme.com/f/justiceforjacobblake,Medical,1,Fundraiser by Julia Jackson : Justice for Jaco...,"Kenosha, WI",Julia Jackson,"$2,252,260","$3,000,000",15,On August 23rd my son was shot multiple times ...,data/images/50669212_1598282954644748_r.jpg
2,https://www.gofundme.com/f/official-navajo-nat...,Medical,2,Fundraiser for Pearline Kirk by Navajo Nation ...,"Window Rock, AZ",Navajo Nation,"$1,770,760","$1,000,000",127,The Navajo Nation COVID-19 Fund has been estab...,data/images/47855360_1589149838795170_r.jpeg
3,https://www.gofundme.com/f/emergenza-covid-ces...,Medical,2,Fundraiser by Gloria Zavatta : Cesvi per l'osp...,Team member,Gloria Zavatta,"€1,506,210","€1,500,000",182,Per aggiornamenti su come stiamo impiegando i ...,data/images/46073102_1583841007387069_r.jpeg
4,https://www.gofundme.com/f/save-nil039s-life,Medical,2,Fundraiser by Berk Atikoğlu : Save Nil's Life!,"Brooklyn, NY",Berk Atikoğlu,"$2,402,980","$2,400,000",182,Save Nil’s Life ! I am starting a campaign for...,data/images/46072754_1583838455417877_r.jpeg
...,...,...,...,...,...,...,...,...,...,...,...
6785,https://www.gofundme.com/f/project-ray-of-hope,Volunteer,331,Fundraiser by Nadia Afrin : Food assistance fo...,"Washington, DC",Nadia Afrin,"$7,035","$12,000",116,We are raising funds to support 200 Bangladesh...,data/images/48097994_1589567702790696_r.jpeg
6786,https://www.gofundme.com/f/to-feed-dailywagers...,Volunteer,332,Fundraiser by Lily Thapa : To Feed Daily-Wager...,"Somerville, MA",Lily Thapa,"$7,034","$20,000",143,Women for Human Rights Group is an organizatio...,data/images/47489998_1587219372600672_r.jpeg
6787,https://www.gofundme.com/f/contribution-to-com...,Volunteer,332,Fundraiser by Mahber Selam : COVID-19 and agai...,"Liendert, Rustenburg, NL",Mahber Selam,"€7,002","€50,000",143,We want mobilize the Eritreans in Tigray and e...,data/images/47496222_1588529991103718_r.jpeg
6788,https://www.gofundme.com/f/covid19-emergency-f...,Volunteer,332,Fundraiser for Niñez con Futuro by Cindy Zetin...,"Sioux Falls, SD",Cindy Zetino,"$6,998","$10,000",108,Thousands of families are currently facing eco...,data/images/48270674_1594729069827005_r.jpg


In [29]:
listInvalidCampaign

[0,
 1,
 1002,
 3,
 4,
 1005,
 1006,
 1007,
 1008,
 1009,
 2925,
 2926,
 5922,
 1923,
 3925,
 3926,
 3927,
 3928,
 3929,
 3930,
 3931,
 3932,
 3933]

In [30]:
lstScrapingError

[1023,
 2028,
 2033,
 2038,
 46,
 2050,
 3069,
 2072,
 5074,
 1093,
 2110,
 6110,
 2116,
 124,
 4129,
 1131,
 1132,
 6161,
 3183,
 4190,
 195,
 1201,
 6202,
 3202,
 202,
 1216,
 3219,
 1233,
 5266,
 6284,
 4290,
 3296,
 5299,
 1301,
 2304,
 305,
 306,
 318,
 3321,
 2334,
 338,
 6343,
 5349,
 5361,
 4379,
 4391,
 2398,
 1409,
 5419,
 5425,
 2428,
 432,
 2444,
 3442,
 5447,
 2479,
 4487,
 489,
 2504,
 1507,
 4519,
 2526,
 5528,
 2545,
 1550,
 3567,
 5571,
 2591,
 593,
 6600,
 2625,
 5641,
 4648,
 5662,
 2674,
 1673,
 1679,
 684,
 4694,
 711,
 2729,
 1749,
 2758,
 2765,
 4790,
 6792,
 2806,
 1803,
 816,
 6821,
 2842,
 842,
 2867,
 1865,
 5874,
 2884,
 883,
 2891,
 2895,
 1901,
 4933,
 933,
 1955,
 955,
 974,
 981,
 1986,
 5992,
 999]

In [36]:
pd.read_csv("data/GFM_data.csv",sep = '\t')

,Url,Category,Position,Title,Location,Organiser,Amount_Raised,Goal,Duration,Text,Image
0,https://www.gofundme.com/f/money-to-buy-hearin...,Medical,0,Fundraiser by Chandan Shivu Kumar : Money to b...,NaN,NaN,$250,$250,236.0,"Hello there, I'm a random guy from India who's...",data/images/44793298_1579175168658123_r.jpeg
1,https://www.gofundme.com/f/justiceforjacobblake,Medical,1,Fundraiser by Julia Jackson : Justice for Jaco...,"Kenosha, WI",Julia Jackson,"$2,252,260","$3,000,000",15.0,On August 23rd my son was shot multiple times ...,data/images/50669212_1598282954644748_r.jpg
2,https://www.gofundme.com/f/official-navajo-nat...,Medical,2,Fundraiser for Pearline Kirk by Navajo Nation ...,"Window Rock, AZ",Navajo Nation,"$1,770,760","$1,000,000",127.0,The Navajo Nation COVID-19 Fund has been estab...,data/images/47855360_1589149838795170_r.jpeg
3,https://www.gofundme.com/f/emergenza-covid-ces...,Medical,2,Fundraiser by Gloria Zavatta : Cesvi per l'osp...,Team member,Gloria Zavatta,"€1,506,210","€1,500,000",182.0,Per aggiornamenti su come stiamo impiegando i ...,data/images/46073102_1583841007387069_r.jpeg
4,https://www.gofundme.com/f/save-nil039s-life,Medical,2,Fundraiser by Berk Atikoğlu : Save Nil's Life!,"Brooklyn, NY",Berk Atikoğlu,"$2,402,980","$2,400,000",182.0,Save Nil’s Life ! I am starting a campaign for...,data/images/46072754_1583838455417877_r.jpeg
...,...,...,...,...,...,...,...,...,...,...,...
6785,https://www.gofundme.com/f/project-ray-of-hope,Volunteer,331,Fundraiser by Nadia Afrin : Food assistance fo...,"Washington, DC",Nadia Afrin,"$7,035","$12,000",116.0,We are raising funds to support 200 Bangladesh...,data/images/48097994_1589567702790696_r.jpeg
6786,https://www.gofundme.com/f/to-feed-dailywagers...,Volunteer,332,Fundraiser by Lily Thapa : To Feed Daily-Wager...,"Somerville, MA",Lily Thapa,"$7,034","$20,000",143.0,Women for Human Rights Group is an organizatio...,data/images/47489998_1587219372600672_r.jpeg
6787,https://www.gofundme.com/f/contribution-to-com...,Volunteer,332,Fundraiser by Mahber Selam : COVID-19 and agai...,"Liendert, Rustenburg, NL",Mahber Selam,"€7,002","€50,000",143.0,We want mobilize the Eritreans in Tigray and e...,data/images/47496222_1588529991103718_r.jpeg
6788,https://www.gofundme.com/f/covid19-emergency-f...,Volunteer,332,Fundraiser for Niñez con Futuro by Cindy Zetin...,"Sioux Falls, SD",Cindy Zetino,"$6,998","$10,000",108.0,Thousands of families are currently facing eco...,data/images/48270674_1594729069827005_r.jpg


In [ ]:
print(mydf.iloc[5].Url)

# Sequential Mode (Online) 

In [ ]:
def scrape_all_urls(start = 0, end = len(mydf), file = 'data/GFM_data.csv'):
    finaldf = pd.DataFrame(columns = headers)
    for i in range(start, end):
        try:
            t = scrape_url(i)
            if(type(t) != str):
                finaldf = pd.concat([finaldf,t], ignore_index = True)
            else:
                print(t)
        except Exception as e:
            print(getattr(e, 'message', repr(e)))
    return finaldf
#             try:
#                 sleep(5)
#                 t = scrape_url(i)
#                 temp_df = pd.concat([temp_df,t], ignore_index = True)
#             except:
#                 print("Cant Scrape Row: "+str(i))
                
#     for i in range(start, end):
#         try:
#             temp_df = scrape_url(i)
#             temp_df.to_csv(file, mode = 'a',sep = '\t', header = False)
#         except Exception('ScrapingError'):
#             print("scraping error for row",i)
#         except Exception("InvalidCampaign"):
#             print("Invalid campaign: Row",i)
#         except:
#             try:
#                 sleep(5)
#                 temp_df = scrape_url(i)
#                 temp_df.to_csv(file, mode = 'a',sep = '\t', header = False)
#             except:
#                 print("Cant Scrape Row: ",i)
        
# #             sleep(5)
# #             print("Scraping url %s" %(i+1))
# #             temp_df = scrape_url(i)
# #             temp_df.to_csv(file, mode = 'a',sep = '\t', header = False)

In [ ]:
# Online Mode
_ = pd.DataFrame(columns = headers)
_.to_csv("data/GFM_data.csv",sep = '\t')

data = scrape_all_urls()

In [ ]:
data

In [ ]:
mydf.head().Url

In [ ]:
mydf = pd.read_csv("data/GFM_data.csv",sep = '\t', index_col = 0)
mydf.reset_index(inplace = True, drop = True)
mydf.tail()

In [ ]:
dropDF = mydf[mydf.Text == "We're sorry, but that campaign cannot be found. Please check the link URL and try again."]
print(dropDF.index)
mydf.drop(dropDF.index,axis = 0, inplace = True)
mydf.reset_index(drop = True, inplace = True)
mydf.to_csv("data/GFM_data.csv",sep = '\t')

In [ ]:
print(mydf.shape)
mydf.head(8)

In [ ]:
print(mydf.Url[3],"\n",mydf.Text[127])

# Rough Work

In [ ]:
row_index = 127

single_row = mydf.iloc[row_index]
url = single_row["Url"]
category = single_row["Category"]
position = single_row["Position"]

page = requests.get(url)
print("URL: ",url)
print("Page: ", page)


In [ ]:
soup = BeautifulSoup(page.text, 'lxml')
# print(soup)
# with open("data/sample_website.html", "w", encoding='utf-8') as file:
#     file.write(str(soup))

In [ ]:
invalidCampaign = soup.find('div', attrs={'class': 'm-campaign-sidebar-notification m-well m-well--sand'})
print(invalidCampaign is not None)

In [ ]:
# Title
soup.find("title").text

In [ ]:
# Organiser
t = soup.find_all("div",{"class":"p-campaign-members"})[0]
# print(t)
print("Organiser: ",t.find_all("div",{"class": "m-person-info-name"})[0].text.replace('\xa0',''))
if(len(t.find_all("div",{"class": "text-small"})) > 2):
    location = t.find_all("div",{"class": "text-small"})[2].text.replace('\xa0','')
else:
    location = t.find_all("div",{"class": "text-small"})[1].text.replace('\xa0','')
print("Location: ",location)
t.find_all("div",{"class": "text-small"})

In [ ]:
# Amount Raised sidebar
t = soup.find("div",{"class":"p-campaign-sidebar"})
amount = t.find("div",{"class": "o-campaign-sidebar-progress-meter m-progress-meter"})
amount.find("h2",{"class":"m-progress-meter-heading"})
print("Amount Raised: ", amount.text.split()[0])
print("Goal: ", amount.text.split()[-2])

print("Number of Donors: ","N/A (yet)")
# t

In [ ]:
# Duration of Fundraising
t = soup.find("div",{"class": "p-campaign-description"})
startDate = ' '.join(t.find("li",{"class": "m-meta-list-item"}).text.split()[1::])
startDate = (pd.to_datetime(startDate)).date()
duration = (pd.datetime.now().date() - startDate).days
print("Start Date: ",startDate,"\nDuration: ",duration)

In [ ]:
# Image
img_container =  soup.find('meta', attrs={'property': 'og:image'})
image_url = img_container['content']

filename = "data/images/"+image_url.split("/")[-1]

# Open the url image, set stream to True, this will return the stream content.
r = requests.get(image_url, stream = True)

# Check if the image was retrieved successfully
if r.status_code == 200:
    # Set decode_content value to True, otherwise the downloaded image file's size will be zero.
    r.raw.decode_content = True
    
    # Open a local file with wb ( write binary ) permission.
#     with open(filename,'wb') as f:
#         shutil.copyfileobj(r.raw, f)
        
    print('Image sucessfully Downloaded: ',filename)
else:
    print('Image Couldn\'t be retreived')

In [ ]:
# All Text
soup.find('div', {'class': "p-campaign-description"})
text = soup.find('div', {'class': "o-campaign-story mt-three-halves"}).text.replace('\xa0','').replace('\n','')
text